In [ ]:
# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas
from sklearn.decomposition import PCA
import numpy
import matplotlib.pyplot as plot
import random
from scipy import sparse
from lightfm import LightFM
from lightfm.evaluation import auc_score

# Load Data

In [ ]:
# list of prefered dataframe names in same order as directory
customers = pd.read_csv("/project/data/raw/olist_customers_dataset.csv")
location = pd.read_csv("/project/data/raw/olist_geolocation_dataset.csv")
orders = pd.read_csv("/project/data/raw/olist_orders_dataset.csv")
order_items = pd.read_csv("/project/data/raw/olist_order_items_dataset.csv")
payments = pd.read_csv("/project/data/raw/olist_order_payments_dataset.csv")
reviews = pd.read_csv("/project/data/raw/olist_order_reviews_dataset.csv")
products = pd.read_csv("/project/data/raw/olist_products_dataset.csv")
sellers = pd.read_csv("/project/data/raw/olist_sellers_dataset.csv")
translation = pd.read_csv("/project/data/raw/product_category_name_translation.csv")

# Merge Order with order item

In [ ]:
location = location.sort_values(by=['geolocation_zip_code_prefix'])

In [ ]:
location = location.groupby(['geolocation_zip_code_prefix','geolocation_city','geolocation_state']).max().reset_index()

In [ ]:
location

In [ ]:
customers.columns

In [ ]:
master_df = orders.merge(order_items, on="order_id").merge(customers, on="customer_id").merge(sellers, on="seller_id").merge(products, on="product_id").merge(translation, on='product_category_name').merge(reviews, on="order_id").merge(location, left_on=['customer_zip_code_prefix', 'customer_city', 'customer_state'], right_on=['geolocation_zip_code_prefix','geolocation_city','geolocation_state'])
master_df = master_df.sort_values(by=['order_purchase_timestamp'])
master_df['order_delivered_customer_date'] =  pd.to_datetime(master_df['order_delivered_customer_date'], format='%Y-%m-%d %H:%M:%S')
master_df['order_estimated_delivery_date'] =  pd.to_datetime(master_df['order_estimated_delivery_date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
master_df.rename(columns ={'geolocation_lat':'customer_geolocation_lat','geolocation_lng':'customer_geolocation_lng'},inplace= True)

In [ ]:
master_df = master_df.merge(location, left_on=['seller_zip_code_prefix', 'seller_city', 'seller_state'], right_on=['geolocation_zip_code_prefix','geolocation_city','geolocation_state'])

In [ ]:
master_df.rename(columns ={'geolocation_lat':'seller_geolocation_lat','geolocation_lng':'seller_geolocation_lng'},inplace= True)
master_df.drop(columns=['geolocation_zip_code_prefix_x','geolocation_city_x','geolocation_state_x','geolocation_zip_code_prefix_y','geolocation_city_y','geolocation_state_y'],inplace= True)

Get only delivered item

In [ ]:
master_df.dropna(subset = ['order_delivered_customer_date'],inplace=True)

In [ ]:
#master_df['is_late'] = master_df['order_delivered_customer_date'] > master_df['order_estimated_delivery_date']

In [ ]:
master_df['day_diff_delivered_estimate'] = (master_df['order_delivered_customer_date'] - master_df['order_estimated_delivery_date']).dt.days


In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

In [ ]:
master_df['distance_between_seller_and_buyer'] = haversine_distance(master_df['customer_geolocation_lat'],master_df['customer_geolocation_lng'],master_df['seller_geolocation_lat'],master_df['seller_geolocation_lng'])

In [ ]:
df = master_df[['distance_between_seller_and_buyer','day_diff_delivered_estimate']]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
sns.set_theme(style="whitegrid")
#sns.scatterplot(data=df, x="distance_between_seller_and_buyer", y="day_diff_delivered_estimate",ax=ax)

sns.kdeplot(
    data=df,
    x="distance_between_seller_and_buyer",
    y="day_diff_delivered_estimate",
    levels=5,
    fill=True,
    alpha=0.6,
    cut=2,
    ax=ax,
)